In [61]:
# VCTK Corpus Path
__CORPUSPATH__ = '/home/podman/datasets/jvs100'

# output path
__OUTPATH__ = '/home/podman/StarGANv2-VC/Data/JVS100'

In [43]:
import os
from scipy.io import wavfile
from pydub import AudioSegment

from pydub import AudioSegment
from pydub.silence import split_on_silence
import os
from tqdm import tqdm

def split(sound):
    dBFS = sound.dBFS
    chunks = split_on_silence(sound,
        min_silence_len = 100,
        silence_thresh = dBFS-16,
        keep_silence = 100
    )
    return chunks

def combine(_src):
    audio = AudioSegment.empty()
    for i,filename in enumerate(os.listdir(_src)):
        if filename.endswith('.wav'):
            filename = os.path.join(_src, filename)
            # print(os.path.exists(filename), filename)
            audio += AudioSegment.from_wav(filename)
    return audio

def save_chunks(chunks, directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    counter = 0

    target_length = 5 * 1000
    output_chunks = [chunks[0]]
    for chunk in chunks[1:]:
        if len(output_chunks[-1]) < target_length:
            output_chunks[-1] += chunk
        else:
            # if the last output chunk is longer than the target length,
            # we can start a new one
            output_chunks.append(chunk)

    for chunk in output_chunks:
        chunk = chunk.set_frame_rate(24000)
        chunk = chunk.set_channels(1)
        counter = counter + 1
        chunk.export(os.path.join(directory, str(counter) + '.wav'), format="wav")

In [65]:
# Source: http://speech.ee.ntu.edu.tw/~jjery2243542/resource/model/is18/en_speaker_used.txt
# Source: https://github.com/jjery2243542/voice_conversion

# speakers = list()
# for i in range(1, 101):
#     speakers.append('jvs{:0>3}'.format(i))
    
speakers = ['jvs016', 'jvs035',  'jvs092',
                     'jvs040', 'jvs067', 'jvs066', 'jvs095',
                     'jvs036', 'jvs090', 'jvs061',
                     'jvs006', 'jvs021', 'jvs042',
                     'jvs003', 'jvs020', 'jvs045', 'jvs073',
                     'jvs047',  'jvs032', 'jvs098']
print(len(speakers))

20


In [5]:
# downsample to 24 kHz

for p in tqdm(speakers):
    directory = __OUTPATH__ + str(p)
    if not os.path.exists(directory):
        audio = combine(__CORPUSPATH__  + str(p))
        chunks = split(audio)
        save_chunks(chunks, directory)

100%|██████████████████████████████████████████████████████████████████████████████████████| 100/100 [10:13<00:00,  6.14s/it]


In [66]:
# get all speakers

data_list = []
for path, subdirs, files in os.walk(__OUTPATH__):
    for name in files:
        if name.endswith(".wav"):
            speaker = path.split('/')[-1]
            if speaker in speakers:
                data_list.append({"Path": os.path.join(path, name), "Speaker": int(speakers.index(speaker)) + 1})
import pandas as pd

data_list = pd.DataFrame(data_list)
data_list = data_list.sample(frac=1)

import random

split_idx = round(len(data_list) * 0.1)

test_data = data_list[:split_idx]
train_data = data_list[split_idx:]

In [67]:
# write to file 

file_str = ""
for index, k in train_data.iterrows():
    file_str += k['Path'] + "|" +str(k['Speaker'] - 1)+ '\n'
print(os.path.exists(__OUTPATH__))
with open(__OUTPATH__ + "/train_list.txt", "w") as f:
    f.write(file_str)

file_str = ""
for index, k in test_data.iterrows():
    file_str += k['Path'] + "|" + str(k['Speaker'] - 1) + '\n'
text_file = open(__OUTPATH__ + "/val_list.txt", "w")
text_file.write(file_str)
text_file.close()

True


'/home/podman/StarGANv2-VC/Data'

AttributeError: module 'posixpath' has no attribute 'pwd'